# Instrumental VEI

In this tutorial, we will attempt to draft an instrumental (seismic) VEI scale. One scale will be based on Reduced Displacement, another will use Energy Magnitude directly, and the third will be based on modelling volcanic events by a single force. First, some background on these.


# 1. Reduced Displacement

Reduced Displacement ($D_R$) is a measurement for comparing the sizes of different seismic signals, such as those corresponding to volcanic eruptions and lava dome collapses.

Seismometers are (usually) velocity transducers - they generate an electric signal (in Volts) that is proportional to the velocity of ground shaking within a certain passband. Outside that passband, the seismometer no longer generates a voltage proportional to ground velocity. This voltage signal has to be digitized (using an analog-to-digital converter built into the datalogger or on a computer at the observatory), converting the signal units from Volts to digitizer "Counts" before it can be examined on a computer. This is a raw seismic signal. To convert this raw seismic signal to a velocity seismogram, we must "remove the instrument response". Then by carefully integrating the velocity seismogram, we will have a displacement seismogram. So we have raw, velocity, and displacement, seismograms.

RSAM is just one of several related downsampled seismic measurements. Some others supported by the `SAM` package are:

<table>
    <tr><th>Acronym</th><th>Full name</th><th>Computed from a:</th><th>Instrument corrected?</th><th>Geometrically corrected?</th></tr>
    <tr><td>RSAM</td><td><b>Real-time (or Raw) Seismic Amplitude Measurement</b></td><td>Raw seismogram</td><td>N</td><td>N</td></tr>
    <tr><td>VSAM</td><td>Velocity Seismic Amplitude Measurement</td><td>Velocity seismogram</td><td>Y</td><td>N</td></tr>
    <tr><td>DSAM</td><td>Displacement Seismic Amplitude Measurement</td><td>Displacement seismogram</td><td>Y</td><td>N</td></tr>
    <tr><td>RSEM</td><td>Raw Seismic Energy Measurement</td><td>Raw seismogram</td><td>N</td><td>N</td></tr>   
    <tr><td>VSEM</td><td>Velocity Seismic Energy Measurement</td><td>Velocity seismogram</td><td>Y</td><td>N</td></tr>
    <tr><td>$V_R$ and $V_{RS}$</td><td>Reduced Velocity</td><td>VSAM</td><td>Y</td><td>Y</td></tr>  
    <tr><td>$D_R$ and $D_{RS}$</td><td><b>Reduced Displacement</b></td><td>DSAM</td><td>Y</td><td>Y</td></tr>   
    <tr><td>$E_R$</td><td><b>Reduced Energy</b></td><td>VSEM</td><td>Y</td><td>Y</td></tr>     
</table>

## 1.1 Correcting for Geometrical Spreading

How do we compare seismograms recorded at different distances from the same volcano? Or from different volcanoes? One thing we can do is to "reduce" the seismograms to a common distance, e.g. 1 km, by correcting for geometrical spreading. This is the idea behind Reduced Displacement. 

Consider a simple half-space velocity model of uniform density. As body waves propagate to greater distances, the wavefront can be thought of as a hemispherical shell, with surface area $2\pi{r}^2$. If energy is not dissipated inelastically or scattered, the energy per unit area must then diminish $1/{r}^2$. And since amplitude is proportional to the square root of energy, the amplitude of body waves diminishes like 1/r. We call this "geometrical spreading".

Similarly, surface waves can be thought of as a circular (or cylindrical) wavefront, whose circumference increases like $2\pi r$, so energy density diminishes like 1/r and surface wave amplitude as $1/\sqrt(r)$.

Reduced Displacement was introduced by Aki & Koyanagi (1981) as a measurement of volcanic tremor amplitude for Kilaeua. Proportional to seismic moment rate, it is the RMS amplitude of a displacement seismogram, corrected for geometrical spreading. 

For body waves:

$D_{R} = rms(U) r$

where r = distance from source to station, U = displacement

Beyond a few wavelengths, surface waves dominate, and Fehler (1983) determined:

$D_{RS} = rms(U) \sqrt{r \lambda}$

In a series of papers McNutt (1994; 2005; 2008) demonstrated scaling relationships for explosive volcanic eruptions between Reduced Displacement and:
- ash column height
- Volcanic Explosivity Index (Newhall and Self, 1982)
- cross-sectional area of volcanic vent

RSAM is generally more useful for seismic field engineers, because it is unfiltered. Reduced Displacement is generally more useful for scientists, as it is a measurement that can be compared from station to station, even on different volcanoes. 

References:

- Aki, K., Koyanagi, R.Y., 1981. Deep volcanic tremor and magma ascent mechanism under Kilauea, Hawaii. J. Geophys. Res. 86, 7095–7110.
- Fehler, M., 1983. Observations of volcanic tremor at Mount St. Helens Volcano. J. Geophys. Res. 88, 3476–3484.
- McNutt, S.R., 1994. Volcanic tremor amplitude correlated with the Volcanic Explosivity Index and its potential use in determining ash hazards to aviation. Acta Vulcanol. 5, 193–196.
- McNutt, S.R., Nishimura, T., 2008. Volcanic tremor during eruptions: Temporal characteristics, scaling and constraints on conduit size and processes. J. Volcanol. Geotherm. Res., 178, 1, 10-18. https://doi.org/10.1016/j.jvolgeores.2008.03.010.
- Newhall, C.G., Self, S., 1982. The volcanic explosivity index (VEI): an estimate of explosive magnitude for historical volcanism. J. Geophys. Res. 87, 1231–1238.

## 1.2 Computing Reduced Displacement

DSAM is just an instrument-corrected version of RSAM, and $D_R$/$D_{RS}$  is just DSAM corrected for geometrical spreading, and so SAM.py was written to support all these different measurements. Here is an algorithm:

1. Load raw seismic data as an ObsPy Stream object.
2. Load station/instrument response metadata from a StationXML file into an Obspy Inventory object.
3. Remove instrument response to convert Stream object to a displacement seismogram.
4. Compute DSAM data from this displacement seismogram. 
6. Define a fixed source location.
7. Compute distance from station to source (done internally using source and inventory)
8. "Reduce" the DSAM object by multiplying by geometrical spreading correction.

# 2. Energy Magnitude

Ampitude measurements, such as RSAM and Reduced Displacement, do not effectively capture the size of an event because some events of equal amplitude may be of very different durations. So it is useful to estimate the energy of the event. Furthermore, because event energy can range of several orders of magnitude, it is useful to define an energy magnitude.

We compute Reduced Energy ($E_R$, units: J) via the equation:

$E_R = 2 \pi r^{2}  \frac{\rho_{E} c_{E}}{A} \int {S^{2} {U(t)}^2} dt$

(Boatwright, 1980; Johnson and Aster, 2005). 

Since this includes a correction for geometrical spreading, we will call it Reduced Energy ($E_{R}$).

The quantity ---- is similar to RSEM (Real-time Seismic Energy Measurement) but computed on a velocity seismogram rather than a raw seismogram, so we call it VSEM.

And then following Hanks and Kanamori (1???), we define Energy-Magnitude ($M_{E}$) as:

(Started doing this in March 2000 at Montserrat Volcano Observatory... lots of complaints!)

$M_{E} = \frac{2}{3} log_{10} E_R - 3.2$

(The factor 3.2 might change depending on velocity structure).

We will now compute $E_{R}$ and $M_{E}$ for the same events we examined in the Reduced Displacement tutorial.

References:

- Boatwright, 1980
- & Boatright, 1995 (original Energy Magnitude definition)
- Johnson & Aster, 2005
- VSEM?
- Hanks and Kanamori, 1980?

## 2.1 Computing Reduced Energy ($E_R$)

DSAM is just an instrument-corrected version of RSAM, and Reduced Displacement is just DSAM corrected for geometrical spreading, and so SAM.py was written to support all these different measurements. Here is an algorithm:

1. Load raw seismic data as an ObsPy Stream object.
2. Load station/instrument response metadata from a StationXML file into an Obspy Inventory object.
3. Remove instrument response to convert Stream object to a displacement seismogram.
4. Compute DSAM data from this displacement seismogram. 
6. Define a fixed source location.
7. Compute distance from station to source (done internally using source and inventory)
8. "Reduce" the DSAM object by multiplying by geometrical spreading correction.

# 3. Single Force

A vertically-directed volcanic explosion can be modelled as a single force using moment tensor representation. Similarly, a directed blast can be modelled as a titled or horizontal single force. Felix Cardozo plans to make some estimates with MTUQ, and just needs instrument-corrected (displacement) seismograms.

# 4. Methodology

## Functions

In [1]:
import os
import sys
#import glob
import numpy as np
import pandas as pd
import obspy
sys.path.append('lib')
import dataframe_tools as DFT
from DFT import wrapper
from SAM import DSAM, VSEM, DR, DRS

from math import log10, floor
def sigfigs(x, n=2):
    if x==0:
        return 0
    else:
        return round(x, n-int(floor(log10(abs(x))))-1)


# Load raw seismic data - and set units accordingly
DATA_DIR = os.path.join('data')
SDS_DIR = os.path.join(DATA_DIR, 'continuous','SDS')
SAM_DIR = os.path.join(DATA_DIR, 'continuous','SAM')
RESPONSE_DIR = os.path.join(DATA_DIR, 'responses')

# Load raw seismic data - and set units accordingly
SDS_DIR = os.path.join('/data', 'SDS')
#SAM_DIR = os.path.join(DATA_DIR, 'continuous','SAM')
RESPONSE_DIR = os.path.join('data', 'responses')

#resultsDF = pd.DataFrame(columns=['Event', 'sum(ER)', 'ME', 'sum(ER_VLP)', 'ME_VLP', 'DR', 'DR_VLP', 'DRS', 'DRS_VLP', 'M_DR', 'M_DR_VLP', 'M_DRS', 'M_DRS_VLP'])

resultsDF = pd.DataFrame(columns=['Event', 'start', 'end', 'duration', 'ML', 'sum(ER)', 'ME', 'DR', 'DRS']) #,  'M_DR',  'M_DRS'])



ModuleNotFoundError: No module named 'DFT'

# 5. Events

## 5.1 Event 1: Boxing Day Collapse 1997, Montserrat

29 months into the eruption of the Soufriere Hills Volcano, Montserrat, part of the crater wall the lava dome was growing within (and overtopping) suddenly collapsed in a landslide, causing a sideways explosion of the lava dome which effectively removed all traces of villages in the southwest quadrant of Montserrat (the villages had been evacuated more than a year earlier). At this time, there were only 2 stations operational, due to months of pyroclastic flows which had destroyed the capital, Plymouth, the airport, numerous other villages, and several seismic stations, and it was much too dangerous to replace them.

In [ ]:
# Event name
eventname1 = 'Boxing Day Collapse 1997'
source1 = {'lat':16.71111, 'lon':-62.17722}
network1 = 'MV'
stationxml1 = os.path.join(RESPONSE_DIR, f'{network1}.xml')
startt1 = obspy.UTCDateTime(1997,12,26,6,40,0)
endt1 = obspy.UTCDateTime(1997,12,26,7,40,0)
trace_ids=['MV.MBWH..SHZ', 'MV.MBLG..SHZ']
wrapper(SDS_DIR, network1, startt1, endt1, stationxml1, resultsDF, eventname1, source1, trace_ids=trace_ids)

In this example, for the regular passband, we get very similar values of $D_R$ and $D_{RS}$ - both around 217 ${cm}^2$. In the VLP passband, the signal is 10 times smaller, which isn't surprising as the only stations available had short-period seismometers.

## 5.2 July 12th 2003 Dome collapse, Montserrat
On July 12th, 2003, over 200 million ${m}^3$ of the lava dome collapse in a series of explosions and pyroclastic flows, over a few hours. Let's compare the $D_R$ (and $D_{RS}$) of this event to the Boxing Day 1997 collapse we just examined. Since this collapse happened mostly down the Tar River Valley, we will pick a source location there, rather than one centred on the lava dome.

In [ ]:
eventname2 = ['2003/07/12 low-medium PDCs', 
              '2003/07/13 large PDC', 
              '2003/07/13 major collapse', 
              '2003/07/13 03:35 explosion',
              '2003/07/13 13:08 explosion',
              '2003/07/14 05:14 explosion',
              '2003/07/15 05:28 explosion']
              
stationxml2 = stationxml1
source2 = {'lat':16.7164, 'lon':-62.1654}  # Tar River

startt2 = [obspy.UTCDateTime(2003,7,12,13,30,0), 
           obspy.UTCDateTime(2003,7,13,0,20,20), 
           obspy.UTCDateTime(2003,7,13,2,0,0), 
           obspy.UTCDateTime(2003,7,13,3,34,0),
           obspy.UTCDateTime(2003,7,13,13,7,0),
           obspy.UTCDateTime(2003,7,14,5,14,0),
           obspy.UTCDateTime(2003,7,15,5,28,0)]
        
endt2 = [obspy.UTCDateTime(2003,7,13,0,0,0), 
         obspy.UTCDateTime(2003,7,13,0,25,0), 
         obspy.UTCDateTime(2003,7,13,4,40,0),
         obspy.UTCDateTime(2003,7,13,3,37,0),
         obspy.UTCDateTime(2003,7,13,13,10,0),
         obspy.UTCDateTime(2003,7,14,5,20,0),
         obspy.UTCDateTime(2003,7,15,5,40,0)]

trace_ids = trace_ids=['MV.MBGH..BHZ', 'MV.MBLG..SHZ', 'MV.MBRY..BHZ', 'MV.MBSS..SHZ', 'MV.MBWH..SHZ']
wrapper(SDS_DIR, network1, startt2, endt2, stationxml2, resultsDF, eventname2, source2, trace_ids = trace_ids)


In [ ]:
'''
#eventname2 = ['2001/07/29 major collapse']
#startt2 = [obspy.UTCDateTime(2001,7,29,0,0,0)]    
#endt2 = [obspy.UTCDateTime(2001,7,31,0,0,0)]   
#eventname2 = ['1997/06/25 collapse']
#startt2 = [obspy.UTCDateTime(1997,6,25,0,0,0)]    
#endt2 = [obspy.UTCDateTime(1997,6,26,0,0,0)]  
#eventname2 = ['2000/03/20 collapse']
#startt2 = [obspy.UTCDateTime(2000,3,19,0,0,0)]    
#endt2 = [obspy.UTCDateTime(2000,3,21,0,0,0)] 
eventname2 = ['2001/03/01 tremor']
startt2 = [obspy.UTCDateTime(2001,3,1,0,0,0)]    
endt2 = [obspy.UTCDateTime(2001,3,2,0,0,0)] 
stationxml2 = stationxml1
source2 = {'lat':16.7164, 'lon':-62.1654}  # Tar River

#race_ids = trace_ids=['MV.MBGH..BHZ', 'MV.MBLG..SHZ', 'MV.MBRY..BHZ', 'MV.MBSS..SHZ', 'MV.MBWH..SHZ']
wrapper(SDS_DIR, network1, startt2, endt2, stationxml2, resultsDF, eventname2, source2)##, trace_ids = trace_ids)
'''

## 3.3 Phreatic eruptions, Whakaari

Whakaari - otherwise known as White Island - is the subaerial tip of a 1600 m high volcano that rises from the seafloor. Tourists arriving by boat and helicopter take a short hike into an open volcanic crater, past fumaroles, to an overlook of the crater lake. Phreatic eruptions are particularly difficult to forecast, as unlike magmatic eruptions, there are no seismic precursors (or indeed any other precuroses) that have been identified. Sadly, this particulary eruption led to the deaths of 22 tourists and guides. Let's compute the $D_R$ (and $D_{RS}$) of this event.


In [ ]:
network3 = 'NZ'
eventname3 = 'Phreatic explosion, Whakaari'
stationxml3 = os.path.join(RESPONSE_DIR, 'NZ.xml')
source3 = {'lat':-37.5217, 'lon':177.185}
startt3 = [obspy.UTCDateTime(2012,8,4,16,52,0),
           obspy.UTCDateTime(2013,8,19,22,23,0),
           obspy.UTCDateTime(2013,10,3,3,35,0),
           obspy.UTCDateTime(2013,10,8,2,5,0),
           obspy.UTCDateTime(2013,10,11,7,9,0),  
           obspy.UTCDateTime(2016,4,27,9,37,0),             
           obspy.UTCDateTime(2019,12,9,1,10,0),
          ]
endt3 = [obspy.UTCDateTime(2012,8,4,17,10,0),  
         obspy.UTCDateTime(2013,8,19,22,41,0),
         obspy.UTCDateTime(2013,10,3,3,51,0),
         obspy.UTCDateTime(2013,10,8,2,23,0),  
         obspy.UTCDateTime(2013,10,11,7,27,0),
         obspy.UTCDateTime(2016,4,27,9,55,0),            
         obspy.UTCDateTime(2019,12,9,1,28,0),
        ]
wrapper(SDS_DIR, network3, startt3, endt3, stationxml3, resultsDF, eventname3, source3)




## 3.4 Sub-Plinian eruption sequence, Redoubt volcano, March 23 - April 4, 2009

Redoubt volcano in Alaska had been in a state of unrest for 2-3 months, with deep-long-period earthquakes, deformation, swarms, and tremor, before explosively erupting at least 18 times between March 23rd and March 28th, 2009. We look only at March 23rd. There were actually 5 major explosive events on this day, which began at 6:38am (18,000 ft), 7:02am (44,000 ft), 08:14am (43,000 ft), 9:39am (43,000 ft), and 12:31pm (49,000 ft). 

The seismic data come from an analog telemetry system, and most channels are heavily contaminated by large interference spikes, and if we don't remove these, we'll end up with $D_R$   of ~10^7 ${cm}^2$ !! It turns out there are only two stations free of this noise, RDN and REF, although these also become contaminated from around 3pm onwards.

The seismograms look clean, so now we'll compute the DSAM data and apply a clip level of 0.01 mm to the displacement seismogram. This might take 1-2 minutes because it is computing DSAM data for 24 hours of multi-channel data.

In [ ]:
network4 = 'AV'
# durations from Helena's table, +2 mins
startt4 = [
    obspy.UTCDateTime(2009,2,15,21,5,0),
    obspy.UTCDateTime(2009,3,23,6,34,0),
    obspy.UTCDateTime(2009,3,23,7,2,0),    
    obspy.UTCDateTime(2009,3,23,8,14,0),     
    obspy.UTCDateTime(2009,3,23,9,38,0),
    obspy.UTCDateTime(2009,3,23,12,30,0),    
    obspy.UTCDateTime(2009,3,24,3,40,0),   
    obspy.UTCDateTime(2009,3,26,16,34,0), 
    obspy.UTCDateTime(2009,3,26,17,24,0),   
    obspy.UTCDateTime(2009,3,27,7,47,0),  
    obspy.UTCDateTime(2009,3,27,8,28,0),  
    obspy.UTCDateTime(2009,3,27,16,39,0),  
    obspy.UTCDateTime(2009,3,28,1,34,0),
    obspy.UTCDateTime(2009,3,28,3,24,0),   
    obspy.UTCDateTime(2009,3,28,7,19,0),  
    obspy.UTCDateTime(2009,3,28,9,19,0),  
    obspy.UTCDateTime(2009,3,28,21,40,0),  
    obspy.UTCDateTime(2009,3,28,23,29,0), 
    obspy.UTCDateTime(2009,3,29,3,23,0),
    obspy.UTCDateTime(2009,4,4,13,58,0)
]
endt4 =   [
    obspy.UTCDateTime(2009,2,15,21,23,0),
    obspy.UTCDateTime(2009,3,23,6,38,0),
    obspy.UTCDateTime(2009,3,23,7,9,0), 
    obspy.UTCDateTime(2009,3,23,8,36,0), 
    obspy.UTCDateTime(2009,3,23,10,2,0),  
    obspy.UTCDateTime(2009,3,23,12,52,0),  
    obspy.UTCDateTime(2009,3,24,3,57,0), 
    obspy.UTCDateTime(2009,3,26,16,37,0),  
    obspy.UTCDateTime(2009,3,26,17,37,0),
    obspy.UTCDateTime(2009,3,27,7,50,0), 
    obspy.UTCDateTime(2009,3,27,8,35,0), 
    obspy.UTCDateTime(2009,3,27,16,49,0), 
    obspy.UTCDateTime(2009,3,28,1,38,0), 
    obspy.UTCDateTime(2009,3,28,3,30,0), 
    obspy.UTCDateTime(2009,3,28,7,23,0),  
    obspy.UTCDateTime(2009,3,28,9,23,0), 
    obspy.UTCDateTime(2009,3,28,21,48,0), 
    obspy.UTCDateTime(2009,3,28,23,40,0), 
    obspy.UTCDateTime(2009,3,29,3,35,0),
    obspy.UTCDateTime(2009,4,4,14,2,0)
]
eventname4 = '2009 Redoubt'
stationxml4 = os.path.join(RESPONSE_DIR, 'RD.xml')
source4 = {'lat':60.4845, 'lon':-152.7392}
wrapper(SDS_DIR, network4, startt4, endt4, stationxml4, resultsDF, eventname4, source4, trace_ids=['AV.RDN..EHZ', 'AV.REF..EHZ'])


## 3.5 Hunga Tonga eruption, Jan 15th, 2022

Our final example is Hunga Tonga, which on January 15th, 2022, exploded spectacularly, destroying most of the island, generating a tsunami, and seismic, pressure and gravity waves that propagated around the globe several times. 

In [ ]:
from obspy.clients.filesystem.sds import Client as sdsclient
eventname5 = 'Hunga Tonga, 2022/01/15'
network5 = 'II'
#mySDSclient = sdsclient(SDS_DIR)
startt5 = obspy.UTCDateTime(2022,1,15,4,15,0)
endt5 = obspy.UTCDateTime(2022,1,15,4,30,0)
stationxml5 = os.path.join(RESPONSE_DIR, 'II.xml')
source5 = {'lat':-20.57, 'lon':-175.38}
#st = mySDSclient.get_waveforms("II", "MSVF", "10", "BHZ", startt5, endt5)
#st.plot(equal_scale=False);
wrapper(SDS_DIR, network5, startt5, endt5, stationxml5, resultsDF, eventname5, source5, pre_filt=[0.01, 0.02, 0.1, 0.2], filter=[0.01, 0.1])


## 6. Results

In [ ]:
resultsDF.to_csv('instrumental_seismic_VEI_raw_results.csv')

In [ ]:
import numpy as np
import pandas as pd
import sys
sys.path.append('lib')
import dataframe_tools as DFT
import importlib
importlib.reload(DFT)

#resultsDF = pd.read_csv('instrumental_seismic_VEI_initial_results.csv') # old version, with 3.318 higher ML
resultsDF = pd.read_csv('instrumental_seismic_VEI_raw_results.csv')
resultsDF = resultsDF.loc[:, ~resultsDF.columns.str.contains('^Unnamed')]
''' from https://www.weather.gov/media/publications/assessments/redoubt.pdf, page 7
11 3/23 06:38 FL180
12 3/23 07:02 FL440
13 3/23 08:14 FL430
14 3/23 09:39 FL430
15 3/23 12:31 FL490
16 3/24 03:41 FL600
17 3/26 16:34 FL220
18 3/26 17:24 FL620
19 3/27 07:47 FL360
20 3/27 08:29 FL490
21 3/27 16:39 FL510
22 3/28 01:35 FL390
23 3/28 03:25 FL500
24 3/28 07:20 FL390
25 3/28 09:20 FL430
26 3/28 21:40 FL170
27 3/28 23:39 FL400
28 3/29 03:23 FL410
'''

def feet2km(ft, n=1):
    feetPerKm = 3281
    return np.round(ft/feetPerKm, n)
    


resultsDF['Hc_km'] = None
resultsDF['V_km3'] = None
resultsDF['VEI'] = None
resultsDF['VEI_H'] = None
resultsDF['VEI_V'] = None
resultsDF.at[0, 'V_km3'] = 0.06
resultsDF.at[3, 'V_km3'] = 0.2
resultsDF.at[9, 'Hc_km'] = 1.8
# Redoubt values from https://www.weather.gov/media/publications/assessments/redoubt.pdf  (table above)
resultsDF.at[11, 'Hc_km'] = feet2km(18000)
resultsDF.at[12, 'Hc_km'] = feet2km(44000)
resultsDF.at[13, 'Hc_km'] = feet2km(43000)
resultsDF.at[14, 'Hc_km'] = feet2km(43000)
resultsDF.at[15, 'Hc_km'] = feet2km(49000)
resultsDF.at[16, 'Hc_km'] = feet2km(60000)
resultsDF.at[17, 'Hc_km'] = feet2km(22000)
resultsDF.at[18, 'Hc_km'] = feet2km(62000)
resultsDF.at[19, 'Hc_km'] = feet2km(36000)
resultsDF.at[20, 'Hc_km'] = feet2km(49000)
resultsDF.at[21, 'Hc_km'] = feet2km(51000)
resultsDF.at[22, 'Hc_km'] = feet2km(39000)
resultsDF.at[23, 'Hc_km'] = feet2km(50000)
resultsDF.at[24, 'Hc_km'] = feet2km(39000)
resultsDF.at[25, 'Hc_km'] = feet2km(43000)
resultsDF.at[26, 'Hc_km'] = feet2km(17000)
resultsDF.at[27, 'Hc_km'] = feet2km(40000)
resultsDF.at[28, 'Hc_km'] = feet2km(41000)
resultsDF.at[30, 'Hc_km'] = 57
resultsDF.at[30, 'V_km3'] = 8.6
resultsDF.at[30, 'DR'] = resultsDF.loc[30, 'DRS']

resultsDF['V'] = resultsDF['V_km3']*1e9
resultsDF['H'] = resultsDF['Hc_km']*1e3

# SCAFFOLD: keep doing for events 19+
#resultsDF['VEI_DR_SM'] = 2.17 * np.log10(resultsDF['DR']) + 0.17 # # Steve's eqn based on his correlation published
#resultsDF['VEI_DR_GT'] = 2.63 * np.log10(resultsDF['DR']) - 1.27 # from correlating Steve's data, below
#resultsDF['VEI_HC_GT'] = 2.5 * np.log10(resultsDF['Hc_km']) + 0.9  # from correlating Steve's data, below

def V2VEI(V, in_km=False):
    f = 0
    if in_km:
        f = 9
    return np.log10(V) - 4.0 + f

def H2VEI(H, in_km=False):
    f = 0
    if in_km:
        f = 3
    VEI_H1 = (H*10**f + 25)/9 # Sahagian et al poster, for VEI 3+
    VEI_H2 = np.log10(H)-1.0+f # my correlation from definition of VEI wikipedia for VEI <=3
    return np.min([VEI_H1, VEI_H2])

def estimate_VEI(df, Hcol='H', Vcol='V', in_km=False): # H in m, V in m3
    df['VEI_V']=None
    df['VEI_H']=None
    for i, row in df.iterrows():
        #print(i, row)
        VEI = []
        VEI_H = []
        VEI_V = []
        if Hcol and row[Hcol]:
            VEI_H.append(H2VEI(row[Hcol], in_km=in_km))
        if Vcol and row[Vcol]:
            VEI_V.append(V2VEI(row[Vcol], in_km=in_km))
        #print(VEI_H, VEI_V)
        if len(VEI)>0:
            df.at[i, 'VEI'] = np.round(np.max(VEI),1)
        if len(VEI_H)>0:
            df.at[i, 'VEI_H'] = np.round(VEI_H,1)
        if len(VEI_V)>0:
            df.at[i, 'VEI_V'] = np.round(VEI_V,1)

estimate_VEI(resultsDF, Hcol='H', Vcol='V')

#resultsDF.drop(['M_DR', 'M_DRS'], axis='columns', inplace=True)
#resultsDF.drop(columns=['M_DR', 'M_DRS'],inplace=True)
#display(resultsDF)
resultsDF.to_csv('instrumental_seismic_VEI_droppedcolumns.csv')
#if 'tremor' in resultsDF.iloc[7,'Event']:
#    resultsDF.drop([7],inplace=True)

resultsDF['log10DR'] = np.round(np.log10(resultsDF['DR']),1)
#resultsDF['log10DRS'] = np.log10(resultsDF['DRS'])
resultsDF['V_km3'] = resultsDF['V_km3'].astype(float)
resultsDF['log10V'] = np.log10(resultsDF['V_km3'])+9
resultsDF['Hc_km'] = resultsDF['Hc_km'].astype(float)
resultsDF['log10H'] = np.round(np.log10(resultsDF['Hc_km'])+3,1)
resultsDF.drop([1, 2, 3],inplace=True) # remove PDCs
resultsDF['ML'] = resultsDF['ML'] + 2.4
resultsDF.at[30, 'ML'] = 5.8
resultsDF.at[30, 'ME'] = np.nan #NaN
resultsDF2 = resultsDF.copy().iloc[0:-1] # remove Hunga Tonga?

display(resultsDF[['Event', 'start', 'ML', 'ME', 'log10DR', 'VEI_V', 'log10H']])

# 7. Magnitude scale correlations

In [ ]:
print('\n\nWITH HUNGA TONGA\n')

DFT.linearRegressMagnitudesBySubclass(resultsDF, subclass_col='subclass', mag_columns=['log10DR', 'ME', 'VEI_H', 'VEI_V'], \
                                      subclasses=[], \
                                      plot=True, print_stats=True)

In [ ]:
print('\n\nWITHOUT HUNGA TONGA\n')

DFT.linearRegressMagnitudesBySubclass(resultsDF2, subclass_col='subclass', mag_columns=['ME', 'log10DR', 'VEI_H', 'VEI_V'], \
                                      subclasses=[], \
                                      plot=True, print_stats=True)

In [ ]:
DFT.fix_slope(resultsDF2, 'ML', 'ME', mfixed=4/3)
DFT.fix_slope(resultsDF2, 'ME', 'ML', mfixed=3/4)
DFT.fix_slope(resultsDF2, 'log10DR', 'ME', mfixed=4/3)
DFT.fix_slope(resultsDF2, 'ME', 'log10DR', mfixed=3/4)
DFT.fix_slope(resultsDF2, 'ML', 'log10DR', mfixed=1.0)
DFT.fix_slope(resultsDF2, 'log10DR', 'ML', mfixed=1.0)

In [ ]:
# fix slope at 1.33 for ME vs ML
DFT.fix_slope(resultsDF2, 'ML', 'log10DR', mfixed=1.0)

In [ ]:
np.log10(10000)*0.11

Which values are reasonable to accept?
- For event 0, there were only short-period seismometers available, so the VLP-band results should be ignored. Since we are at local distances, we expect body waves to dominate, so we choose $D_R$ over $D_{RS}$ and accept 218 ${cm}^2$.
- For event 1, while broadband seismometers were available, the VLP-band results are lower. Again, we prefer $D_R$ over $D_{RS}$, since we are at local distances, so we accept 342 ${cm}^2$.
- For event 2, while the VLP bands show higher results, Whakaari is a small island, so the VLP energy likely comes from ocean noise. Again, we are at the local scale, so we accept 18.8 ${cm}^2$.
- For event 3, Redoubt is coastal, and we are at local distances, so we choose $D_R$, and accept 64 ${cm}^2$.
- For event 4, station MSVF is about 760 km from Hunga Tonga, so surface waves dominate, and most of the signal is between 0.01 and 0.6 Hz. Is the $D_{RS}^{VLP}$ value of 39,000 ${cm}^2$ reasonable? 

Clearly, we must be very careful applying the same measurements at local, regional, and global scales!

#### CAVEAT: We ignored inelastic attenuation in this analysis. Including it would make Reduced Displacement estimates even higher!

Finally, there is a sort of "magnitude scale" for volcanic eruptions called the Volcanic Explosivity Index (VEI), which similar to earthquake magnitude scales, runs from about 1 to 8. However, VEI is not an instrumental measurement, and instead is estimated from the volume of deposits found, and the height an ash column reaches, and other things. Hunga Tonga was estimated by different researchers to be either a VEI=5 or VEI=6 eruption, so if we fix that at VEI=5.5 and throw caution to the wind, here are instrumental VEI estimates for the other events above too:

# McNutt & Nishimura [2008] dataset #

In [ ]:
import os
import numpy as np
import sys
sys.path.append('lib')
import dataframe_tools as DFT
import pandas as pd
import importlib
importlib.reload(DFT)
mcnuttDF = pd.read_csv(os.path.join(os.getenv('HOME'), 'Dropbox/BRIEFCASE/SSA2024/mcnutt2008.csv'))
mcnuttDF['Duration'] = mcnuttDF['Duration'] * 3600 # hours to seconds
if 'Height' in mcnuttDF.columns:
    mcnuttDF['H'] = mcnuttDF['Height'] * 1000 # km to m
    mcnuttDF['V'] = mcnuttDF['V'] * 1e6 # 10^6m3 to m3
    mcnuttDF.drop(columns=['Height'],inplace=True)

estimate_VEI(mcnuttDF, Hcol='H', Vcol='V')
#display(mcnuttDF)
#mcnuttDF = mcnuttDF.replace({None: np.nan, NaN: np.nan})
#mcnuttDF.iloc[14]['V']
#print(mcnuttDF.isna())

mcnuttDF['log10DR'] = np.round(np.log10(mcnuttDF['DR']),1)
mcnuttDF['log10V'] = np.round(np.log10(mcnuttDF['V']),1)
mcnuttDF['log10H'] = np.round(np.log10(mcnuttDF['H']),1)
mcnuttDF_without_smallest= mcnuttDF[mcnuttDF['log10V']>4.0]
DFT.linearRegressMagnitudesBySubclass(mcnuttDF_without_smallest, mag_columns=['log10DR', 'log10H', 'log10V', 'VEI', 'VEI_H', 'VEI_V'], \
                                      subclasses=[], \
                                      plot=True, print_stats=True)

display(mcnuttDF[['Volcano', 'Date', 'log10DR', 'VEI_V', 'log10H']])

In [ ]:
DFT.fix_slope(mcnuttDF, 'VEI', 'log10DR', mfixed=0.46)

In [ ]:
mcnuttDF_small = mcnuttDF[mcnuttDF['log10DR']<3.05]
DFT.fix_slope(mcnuttDF_small, 'VEI', 'log10DR', mfixed=0.46)

In [ ]:
DFT.fix_slope(mcnuttDF_small, 'log10DR', 'VEI',  mfixed=1.0/0.46)

In [ ]:
DFT.fix_slope(mcnuttDF_small, 'log10H', 'log10DR', mfixed=1.8)
DFT.fix_slope(mcnuttDF_small, 'log10DR', 'log10H', mfixed=1.0/1.8)
#DFT.fix_slope(mcnuttDF_small, 'log10DR', 'VEI_V', mfixed=1.0)

In [ ]:
DFT.linregress(mcnuttDF, 'log10V', 'VEI', plot=True, print_stats=True)
mcnuttDF_small = mcnuttDF[mcnuttDF['log10V']>3.05]
DFT.linregress(mcnuttDF_small, 'log10V', 'VEI', plot=True, print_stats=True)

In [ ]:
DFT.linregress(mcnuttDF, 'log10H', 'VEI', plot=True, print_stats=True)
mcnuttDF_small = mcnuttDF[mcnuttDF['log10H']>3.05]
DFT.linregress(mcnuttDF_small, 'log10H', 'VEI', plot=True, print_stats=True)
mcnuttDF_small = mcnuttDF[mcnuttDF['log10H']<4.2]
DFT.linregress(mcnuttDF_small, 'log10H', 'VEI', plot=True, print_stats=True)

In [ ]:
DFT.linregress(mcnuttDF, 'log10DR', 'log10H', plot=True, print_stats=True)
mcnuttDF_small = mcnuttDF[mcnuttDF['log10DR']<3.05]
#mcnuttDF_small = mcnuttDF_small[mcnuttDF_small['log10Height']>3.2]
DFT.linregress(mcnuttDF_small, 'log10DR', 'log10H', plot=True, print_stats=True)

# Try combining the McNutt & Nishimura with my quantitative measurements

In [ ]:
print(resultsDF.columns)
print(mcnuttDF.columns)

In [ ]:
resultsA = resultsDF[['DR', 'VEI', 'log10DR', 'log10V', 'log10H', 'VEI_H', 'VEI_V']]
print(resultsA)
mcnuttA = mcnuttDF[['DR', 'VEI', 'log10DR', 'log10V', 'log10H', 'VEI_H', 'VEI_V']]
print(mcnuttA)

In [ ]:
combinedA = pd.concat([resultsA,mcnuttA])
display(combinedA)

In [ ]:
DFT.linearRegressMagnitudesBySubclass(combinedA, mag_columns=['log10DR', 'log10H', 'log10V', 'VEI', 'VEI_H', 'VEI_V'], \
                                      subclasses=[], \
                                      plot=True, print_stats=True)

In [ ]:
c = combinedA['VEI_V'] - combinedA['log10DR']
print(c)
print(c.mean())
print(c.median())

# Try analyzing the Redoubt 2009 data I processed with results from McNutt et al 2013 and Buurman et al 2013

In [ ]:
redoubtDF = pd.read_csv(os.path.join(os.getenv('HOME'), 'Developer/skience2024_GTplus/02 Volcano Monitoring/redoubt2009.csv'))
redoubtDF['ME_s'] = np.round(2/3*np.log10(redoubtDF['S_Eng']) - 3.2,1)
redoubtDF['ME_a'] = np.round(2/3*np.log10(redoubtDF['P_Eng']) - 3.2,1)
redoubtDF['log10A_s'] = np.log10(redoubtDF['S_Amp'])
redoubtDF['log10A_a'] = np.log10(redoubtDF['P_Amp'])
totalV = 1e8 # m3
totalE = redoubtDF['sum(ER)'].sum()
redoubtDF['V_ER'] = redoubtDF['sum(ER)']/totalE * totalV
redoubtDF['log10VER'] = np.log10(redoubtDF['V_ER'])
redoubtDF['VEI_Vs'] = np.round(redoubtDF['log10VER'] - 4,1)
totalEA = redoubtDF['P_Eng'].sum()
redoubtDF['V_ERA'] = redoubtDF['P_Eng']/totalEA * totalV
redoubtDF['log10VERA'] = np.log10(redoubtDF['V_ERA'])
redoubtDF['VEI_Va'] = np.round(redoubtDF['log10VERA'] + np.log10(totalE/totalEA),1)
#display(mergedDF[['Event', 'start', 'ML', 'ME', 'log10DR', 'VEI_V', 'log10H', 'ME_s', 'ME_a', 'VEI_Vs', 'VEI_Va']])
mergedDF = pd.merge(redoubtDF, resultsDF, on="Event", suffixes=['_x', None])
mergedDF.loc[:,~mergedDF.columns.str.endswith('_x')]
#display(mergedDF)
display(mergedDF[['Event', 'start', 'ML', 'ME', 'log10DR', 'log10H', 'ME_s', 'ME_a', 'VEI_Vs', 'VEI_Va']])
#display(redoubtDF)
#print(redoubtDF.columns)

In [ ]:
redoubtDF2 = redoubtDF.iloc[1:]
redoubtDF2 = redoubtDF.copy()
#redoubtDF2.drop(0, inplace=True)
redoubtDF2.drop(8, inplace=True)
DFT.linearRegressMagnitudesBySubclass(redoubtDF2, mag_columns=['ME_seismic', 'ME_acoustic', 'ME', 'ML', 'log10SAmp', 'log10PAmp', 'VEI_seismic', 'VEI_acoustic'], \
                                      subclasses=[], \
                                      plot=True, print_stats=True)